# KOL Ascension Log Wrapper (v1)
_A Styling Project by Aaron M. (AKA Captain Scotch, #437479)_

Hey all! This is a wrapper meant to isolate and analyze Ascension logs via Python, formatting everything into a clean Excel wrapper. Originally, this was simply an overlay of [CKB's run log parser in ASH](https://kolmafia.us/showthread.php?22963-RunLogSummary), which is fantastic, but as I'm more of a Python dev this quickly enlarged into a broader project that turned into a personal goal to build a Python log parser. That's in development right now! I tried to make it as clean as possible; let me know if you have any questions.

In [1]:
import pandas as pd
import requests

In [24]:
# Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
kolmafDir = "/home/tbh/parser/parser/KoL",
outputDir = "/home/tbh/parser/parser/KoL",
kolName   = 'threebullethamburgler'
runDates  = ['20200510','20200511','20200512','20200513']
runNotes  = 'blah blah blah blah /n blah blah blah BLAH /n blah blah, blah blah, Baaaaaah'

In [212]:
# Read in the last parsed run via RunLogSum.ash, for testing purposes
oldRun = pd.read_csv('{}data/{}-runlog_0095.txt'.format(kolmafDir,kolName),sep='\t')

In [9]:
# These are, all things considered, extremely small files. So I just read the whole
#   thing into memory so that it's all accessible without finicky itertools stuff.

# Populate our session dictionary
dDict = {}

for day, session in enumerate(runDates):
    # Python is zero-indexed, so we increment day by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    dDict[day+1] = open('/home/tbh/parser/parser/KoL/sessions/{}_{}.txt'.format(kolName.replace(" ","_"), session)).read()
    
    # Remove everything before Valhalla & after freeing Ralph
    if day == 0:
        start = dDict[1].find('Welcome to Valhalla!')
        dDict[1] = dDict[1][start:]
        
    else:
        end   = dDict[day+1].find('Freeing King Ralph')
        dDict[day+1] = dDict[day+1][:end]

# Also, to minimize server hits, grab a few files from the Mafia SVN data
mafCombats = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/combats.txt')
mafMonster = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/monsters.txt')
mafMods    = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/modifiers.txt')

In [10]:
findString = 'dayc'
buffer     = 3
day        = 2

# simple print w/ buffer for finding things within the logs
for ct, x in enumerate(dDict[day].split(sep='\n')):
    if findString.lower() in x.lower(): 
        print('===========\n'+
              '\n'.join(dDict[day].split(sep='\n')[(ct-buffer):(ct+buffer)])+
              '\n===========\n')

Use 2 ten-leaf clover
You acquire disassembled clover (2)

Visiting the Boxing Daycare

[139] Boxing Daycare


Visiting the Boxing Daycare

[139] Boxing Daycare
Encounter: Have a Boxing Daydream
You acquire an item: punch-drunk punch

cast 1 CHEAT CODE: Invisible Avatar
You acquire an effect: Invisible Avatar (10)

Visiting the Boxing Daycare
You acquire an item: blooper ink


After Battle: You gain 8 Wizardliness
After Battle: Plunderous Toyger cracks his knuckles and looks around for something to steal.

Visiting the Boxing Daycare

[178] The Daily Dungeon (Room 1)


equip pants tinsel tights

Visiting the Boxing Daycare

[203] Boxing Daycare


Visiting the Boxing Daycare

[203] Boxing Daycare
Encounter: Visit the Boxing Day Spa
You acquire an effect: Muddled (100)

After Battle: You gain 3 Magicalness
After Battle: You gain 6 Cheek

Visiting the Boxing Daycare
You acquire super deluxe mushroom (5)




In [11]:
def monsterLocation(mon):
    ''' Sub-function that uses KOLMafia's combats directory to ascertain
        where a monster is likely to occur (used for establishing wanderer
        likelihood in the absence of full adventure parsing; was used for
        initial tests of digitize capture). Eventually will use this for
        full error-checking of the parser. '''
    
    locs = []
    
    mon = str(mon).strip()
    
    # Read in combats.txt and reformat it via split
    for line in mafCombats.text.split('\n'):
        if   mon in line: 
            locs = locs + [line.split('\t')[0]]
            
        # For some reason, Camel's Toe shows up as "The" Camel's Toe when
        #   you genie wish; this checks for and removes extraneous "The"
        elif mon.lower().replace('the ','') in line.lower():
                locs = locs + [line.split('\t')[0]]
    
    if locs == []: locs = ['FOOTAGE NOT FOUND']
        
    return locs
    
monsterLocation('Green Ops Soldier')

['The Battlefield (Frat Uniform)']

In [12]:
def extractWanderers(sessionLogs):
    ''' Quick yet annoying function to attempt to ascertain wanderers. The 
        Digitize parser (unfortunately) doesn't work particularly well yet;
        when ascertaining digitize usage I usually just look over the print &
        mental math for the spreadsheet. The enamorang parser works better,
        which is good, since it's currently the only standard "choice" 
        wanderer as of 1/19. 
        
        Features to add: 
        
            - include detection of vote wanderers 
            - once the XML turn parser is done, revise this to reference it 
            - (you basically fuckin did that already in this hacky POS) '''
    
    wandererDict = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        wandererDict[day] = {}
        
        # Tracking digitized monsters is a pain in the ass. Luckily, simply
        #   extracting who they are can be pretty easy with some rough 
        #   nested loops.
        
        digiMons = []
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'casts DIGITIZE!' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                digiMons = digiMons + [monName]
    
        for mon in digiMons:
            
            # Now that we have that list, we want to figure out how many times
            #   they were fought. To do this, we'll need to compare the zone
            #   they originated from to the zones they have been found in.
            #   This method is meh, but kind of works, in the absence of Mafia 
            #   saving intro messages for digitized monsters. 
            
            monLoc = monsterLocation(mon)
            
            # So, cool unintentional thing here; when Mafia logs Witchess, it
            #   totally messes up the turn/combat statement, because it treats
            #   it differently. This is great! It means digitized Witchess
            #   pieces are super easy to find.
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        print('  Could not parse {}'.format(data))
    
        monLOVs = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'uses the LOV Enamorang' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                        
                        try: 
                            turn = int(dataSplit[0][1:])
                            loct = dataSplit[1]
                        except:
                            turn = -999
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                monLOVs[turn] = monName
                
        for turn, mon in monLOVs.items():
            
            # Similar process for above, with a few small items. Here, I am
            #   checking turn differential to make positively sure it is the
            #   enamorang'd monster; it has to be 14 turns! 
            
            monLoc = monsterLocation(mon)
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        foundTurn = int(data[0][1:])
                        currLoct = data[1]
                        # For testing
                        #print('Turn {}: fought {} @ {}.'.format(foundTurn,mon,loct))
                    except:
                        # Warn that something was unparsable 
                        print('  Could not parse {}'.format(data))
                    
                    if turn != foundTurn:
                        wandererDict[day][mon] = {'Type':'Enamorang',
                                         'Location':currLoct,
                                         'Turn':foundTurn}
    
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wandererDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wandererDict.keys()],sort=False).T
    
extractWanderers(dDict)

Empty DataFrame
Columns: []
Index: []

In [13]:
def cleanNames(monName):
    ''' This won't be much of an issue going forward, but
        the Intergnat had an annoying habit of modifying 
        monster names. This attempts to clean that particular 
        detritus to extract pure names. I'm assuming this
        will come back in other forms as well.
        
        Features to add: 
        
            - parse OCRS modifiers 
            - parse "Yes, Can Has" effect'''
    
    # All the strings added to monster names. Haven't added 
    #   OCRS modifiers yet, but intend to. Also would like 
    #   to include rewriting the can has skill, in case it
    #   comes up through the crazy horse, but haven't yet
    
    gnatList = ['NAMED NEIL', 'ELDRITCH HORROR ', 'AND TESLA',
                'WITH BACON!!!', 'WITH SCIENCE']
    
    ocrsMods = []
    
    replList = gnatList + ocrsMods
    
    for repl in replList:
        monName = monName.replace(repl,'')
        
    # Insert renaming for that one cheeseburger skill here
    
    return monName.strip()

In [14]:
def extractWishes(sessionLogs):
    ''' Quick function to ascertain wish usage '''
    
    wishDict = {}
    
    for day in sessionLogs.keys():
        
        wishDict[day] = {}
        
        # Wishes aren't quite as easy as pulls, which are easily snagged
        #   on one line with newline split logic, but they're certainly 
        #   easier than digitizations.
        
        # Combat parsing, utilizing the combat to pull monsters out
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "genie summoned monster" in row:
                turn = row[(row.find('[')+1):row.find(']')]
                wishMon = sessionLogs[day].split('\n')[ct+1].replace('Encounter: ','')
                wishMon = cleanNames(wishMon)
                wishDict[day][ct] = {'Type':'Fight',
                                     'Details':wishMon,
                                     'Turn':int(turn)}
        
        # Extra parsing, using the actual wish URL to fill others
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "&wish=" in row:
                wishString = row[(row.find('&wish=')+6):]
                try: 
                    
                    # Note we have to use ct+2 due to session logs
                    #   taking 2 lines to get from URL to fight. If
                    #   this ever changes, will need to change.
                    
                    wishDict[day][ct+2]['Type']
                    
                except:
                    
                    wishDict[day][ct] = {'Type':'Non-Fight',
                                         'Details':wishString,
                                         'Turn':'?'}

        # Rename the wishes from line # to wish #
        for i, val in enumerate(wishDict[day]):
            wishDict[day][i+1] = wishDict[day].pop(val)
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wishDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wishDict.keys()]).T

extractWishes(dDict)

Empty DataFrame
Columns: []
Index: []

In [15]:
def extractPulls(sessionLogs):
    ''' Quick function to extract normal run pulls into a table. '''
    
    pullDict = {}
    
    for day in sessionLogs.keys():
        
        # Find all your pulls; relies on sessions storing them as 'pull: '
        pulls = [i[6:] for i in sessionLogs[day].split('\n') if i.startswith('pull: ')]
        
        # Split out pulls into a 20 item list, for formatting
        numbs = [int(i[0:2].strip()) for i in pulls]
        pullDict[day] = []
        for count, pull in enumerate(pulls):
            pullDict[day] = pullDict[day] + [pull[1:].strip()]*numbs[count]
        
        if len(pullDict[day]) < 20:
            pullDict[day] = pullDict[day] + ['']*(20-len(pullDict[day]))
    
    # Renaming for simplicity/ease of pasting; 
    pullsOut = pd.DataFrame(pullDict).rename(
        index  = lambda x: '#{}'.format(x+1),
        columns= lambda x: 'Day #{}'.format(x))
    
    return pullsOut

extractPulls(dDict)

,Day #1,Day #2,Day #3,Day #4
#1,cursed pony keg,,,
#2,,,,
#3,,,,
#4,,,,
#5,,,,
#6,,,,
#7,,,,
#8,,,,
#9,,,,
#10,,,,


In [16]:
def goblinParser(sessionLogs):
    ''' Figure out where/when the user encountered sausage goblins.
        This SHOULD NOT be used as a spading tool, as it does not
        properly track important things like how much meat was in
        the grinder and how many turns the grinder was used. It's
        just for me to think about where I dropped goblins in-run '''
    
    goblinDict = {}
    gobNum = 0
    
    for day in sessionLogs.keys():
        
        # I am keeping this a list in case we eventually find a 
        #   sausage goblin boss...
        
        for mon in ['sausage goblin']:
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    gobNum = gobNum + 1
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        # print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        # Gotta show that error!
                        print('  Could not parse {}'.format(data))
                    
                    goblinDict[gobNum] =      {'Turn': turn, 
                                               'Location': loct,
                                               'Day': day}
    
        
    return pd.DataFrame.from_dict(goblinDict).T
                        
goblinParser(dDict)

,Turn,Location,Day
1,1,The Spooky Forest,1
2,2,The Haunted Kitchen,1
3,14,The Spooky Forest,1
4,29,The Haunted Bathroom,1
5,45,The Haunted Bathroom,1
6,66,The Haunted Bathroom,1
7,91,The Outskirts of Cobb's Knob,1
8,91,The Outskirts of Cobb's Knob,1
9,91,The Outskirts of Cobb's Knob,1
10,91,The Outskirts of Cobb's Knob,1


## Unfinished functions, to be added over time

In [ ]:
def buffSelection(sessionLogs):
    ''' Parse out the IOTM buffs selected in the selected run, by day '''

In [ ]:
def itemSelection(sessionLogs):
    ''' Parse out the IOTM items selected in the selected run, by day '''

In [ ]:
def horseBuffSpading(sessionLogs):
    ''' Find out what buffs you get from the crazy horse, for a simulation
        project I'm working on to establish bounds on crazy horse value '''


In [103]:
import re
import pandas as pd
def pizzaCube(sessionLogs):
    ''' Quick function to extract normal run pulls into a table. '''
    kolmafDir = "/home/tbh/parser/parser/KoL",
    outputDir = "/home/tbh/parser/parser/KoL",
    kolName   = 'threebullethamburgler'
    runDates  = ['20200510','20200511','20200512','20200513']
    dDict = {}
    pizzaDict = {}
    pizzaNum = 0
    # Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
    for day, session in enumerate(runDates):
        # Python is zero-indexed, so we increment day by 1 here. 
        #   Also replace spaces w/  for playername.

        dDict[day+1] = open('/home/tbh/parser/parser/KoL/sessions/{}_{}.txt'.format(kolName.replace(" ","_"), session)).read()

        # Remove everything before Valhalla & after freeing Ralph
        if day == 0:
            start = dDict[1].find('Welcome to Valhalla!')
            dDict[1] = dDict[1][start:]

        else:
            end   = dDict[day+1].find('Freeing King Ralph')
            dDict[day+1] = dDict[day+1][:end]
    
        tuples = re.findall(r'\npizza ([^,]+), ([^,]+), ([^,]+), (.+)\n', dDict[day+1])
        text = re.findall(r'eat 1 diabolic pizza\n(.*?)You gain 3 Fullness', dDict[day+1],re.DOTALL)
        for var in range(len(tuples)):
                pizzaDict[pizzaNum] =    {'Day': day+1,
                                        'Ingred1': tuples[var][0], 
                                        'Ingred2':  tuples[var][1],
                                        'Ingred3':  tuples[var][2],
                                        'Ingred4':  tuples[var][3],
                                        'Results': text[var].replace('\n',',')
                                        }
                pizzaNum = pizzaNum + 1
                
    return pd.DataFrame.from_dict(pizzaDict).T

pizzaCube(dDict)

,Day,Ingred1,Ingred2,Ingred3,Ingred4,Results
0,1,Middle of the Road&trade; brand whiskey,PB&J with the crusts cut off,Purple Beast energy drink,PB&J with the crusts cut off,"You gain 135 Strongness,You gain some Muscle p..."
1,1,antique packet of ketchup,snifter of thoroughly aged brandy,disintegrating quill pen,antique packet of ketchup,"You gain 526 Meat,You gain 12 Adventures,You g..."
2,1,hellion cube,Gnollish autoplunger,hot katana blade,very small red dress,"You acquire an item: cigarette lighter,You acq..."
3,2,blood-faced volleyball,handful of nuts and berries,PB&J with the crusts cut off,handful of nuts and berries,"Your familiar gains a pound: Hawkingotron, the..."
4,2,handful of sand,glob of undifferentiated tissue,handful of sand,pie man was not meant to eat,"You acquire an item: sonar-in-a-biscuit,You ac..."
5,2,pie man was not meant to eat,oversized pizza cutter,loaf of soda bread,orcish nailing lube,"You gain 437 Meat,You gain 9 Adventures,You ga..."
6,2,anticheese,pie man was not meant to eat,Middle of the Road&trade; brand whiskey,Middle of the Road&trade; brand whiskey,"You acquire goat cheese (3),You gain 12 Advent..."
7,2,snifter of thoroughly aged brandy,snifter of thoroughly aged brandy,photoprotoneutron torpedo,glob of undifferentiated tissue,"You acquire an item: Duskwalker syringe,You ac..."
8,2,surprisingly capacious handbag,photoprotoneutron torpedo,PB&J with the crusts cut off,scroll of drastic healing,"You acquire an item: cigarette lighter,You acq..."
9,3,dusty bottle of great Zinfandel,imitation White Russian,razor-sharp can lid,Tears of the Quiet Healer,"You acquire an item: unnamed cocktail,You acqu..."
